Steps to run notebook:
1. Create a conda env with python3.8 (Empty conda env)
```
conda create --name snowml python=3.8
```
2. Activate conda env
```
conda activate snowml
```
3. Install conda pkg
```
conda install snowflake-ml-python 
# or local build if there are changes in SnowML lib you need: bazel build //snowflake/ml:wheel
# then do pip install {built pkg}
```
4. Install jupyter notebook
```
conda install jupyter
```
5. Start notebook
```
jupyter notebook
```

## Feature Store Example with Time Series Features
This notebook demonstrates advanced feature store usage with time series features. 
It will compute features from NY taxi trip data and demonstrate connections between training and prediction.
The reference example by Databricks is here: https://docs.databricks.com/en/_extras/notebooks/source/machine-learning/feature-store-with-uc-taxi-example.html#feature-store/feature-store

## Setup UI and Auto Import

In [52]:
# Scale cell width with the browser window to accommodate .show() commands for wider tables.
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### [Optional 1] Import from local code repository

In [66]:
import sys
import os

# Simplify reading from the local repository
cwd=os.getcwd()
REPO_PREFIX="snowflake/ml"
LOCAL_REPO_PATH=cwd[:cwd.find(REPO_PREFIX)].rstrip('/')

if LOCAL_REPO_PATH not in sys.path:
    print(f"Adding {LOCAL_REPO_PATH} to system path")
    sys.path.append(LOCAL_REPO_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### [Optional 2] Import from installed snowflake-ml-python wheel

In [65]:
import sys

sys.path.insert(0, '/tmp/snowml')

## Prepare demo data

In [53]:
import importlib
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F, types as T
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark.types import DateType, TimeType, _NumericType, TimestampType
import datetime


In [115]:
session = Session.builder.configs(SnowflakeLoginOptions()).create()

In [116]:
source_df = session.table("SNOWML_FEATURE_STORE_TEST_DB.TEST_DATASET.yellow_tripdata_2016_01")

source_df = source_df.select(
    [
        "TRIP_DISTANCE", 
        "FARE_AMOUNT",
        "PASSENGER_COUNT",
        "PULOCATIONID",
        "DOLOCATIONID",
        F.cast(source_df.TPEP_PICKUP_DATETIME / 1000000, TimestampType()).alias("PICKUP_TS"),
        F.cast(source_df.TPEP_DROPOFF_DATETIME / 1000000, TimestampType()).alias("DROPOFF_TS"),
    ]).filter("DROPOFF_TS >= '2016-01-01 00:00:00' AND DROPOFF_TS < '2016-01-03 00:00:00'")
source_df.show()

-------------------------------------------------------------------------------------------------------------------------------------
|"TRIP_DISTANCE"  |"FARE_AMOUNT"  |"PASSENGER_COUNT"  |"PULOCATIONID"  |"DOLOCATIONID"  |"PICKUP_TS"          |"DROPOFF_TS"         |
-------------------------------------------------------------------------------------------------------------------------------------
|3.2              |14.0           |1                  |48              |262             |2016-01-01 00:12:22  |2016-01-01 00:29:14  |
|1.0              |9.5            |2                  |162             |48              |2016-01-01 00:41:31  |2016-01-01 00:55:10  |
|0.9              |6.0            |1                  |246             |90              |2016-01-01 00:53:37  |2016-01-01 00:59:57  |
|0.8              |5.0            |1                  |170             |162             |2016-01-01 00:13:28  |2016-01-01 00:18:07  |
|1.8              |11.0           |1                  |161    

## Create FeatureStore Client

Let's first create a feature store client.

We can pass in an existing database name, or a new database will be created upon the feature store initialization.

In [117]:
DEMO_DB = "FS_TIME_SERIES_EXAMPLE"
session.sql(f"DROP DATABASE IF EXISTS {DEMO_DB}").collect()  # start from scratch
session.sql(f"CREATE DATABASE IF NOT EXISTS {DEMO_DB}").collect()
session.sql(f"CREATE OR REPLACE WAREHOUSE PUBLIC WITH WAREHOUSE_SIZE='XSMALL'").collect()

fs = FeatureStore(
    session=session, 
    database=DEMO_DB, 
    name="AWESOME_FS", 
    default_warehouse="PUBLIC",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

## Create and register new Entities

In [118]:
trip_pickup = Entity(name="trip_pickup", join_keys=["PULOCATIONID"])
trip_dropoff = Entity(name="trip_dropoff", join_keys=["DOLOCATIONID"])
fs.register_entity(trip_pickup)
fs.register_entity(trip_dropoff)
fs.list_entities().to_pandas()

,NAME,JOIN_KEYS,DESC
0,TRIP_DROPOFF,DOLOCATIONID,
1,TRIP_PICKUP,PULOCATIONID,


## Define feature pipeline
We will compute a few time series features in the pipeline here.
Before we have *__value based range between__* in SQL, we will use a work around to mimic the calculation (NOTE: the work around won't be very accurate on computing the time series value due to missing gap filling functionality, but it should be enough for a demo purpose)

We will define two feature groups:
1. pickup features
    - Mean fare amount over the past 2 and 5 hours
2. dropoff features
    - Count of trips over the past 2 and 5 hours

### This is a UDF computing time window end
We will later turn these into built in functions for feature store

In [119]:
@F.pandas_udf(
    name="vec_window_end",
    is_permanent=True,
    stage_location=session.get_session_stage(),
    packages=["numpy", "pandas", "pytimeparse"],
    replace=True,
    session=session,
)
def vec_window_end_compute(
    x: T.PandasSeries[datetime.datetime],
    interval: T.PandasSeries[str],
) -> T.PandasSeries[datetime.datetime]:
    import numpy as np
    import pandas as pd
    from pytimeparse.timeparse import timeparse

    time_slice = timeparse(interval[0])
    if time_slice is None:
        raise ValueError(f"Cannot parse interval {interval[0]}")
    time_slot = (x - np.datetime64('1970-01-01T00:00:00')) // np.timedelta64(1, 's') // time_slice * time_slice + time_slice
    return pd.to_datetime(time_slot, unit='s')

### Define feature pipeline logics

In [120]:
from snowflake.snowpark import Window
from snowflake.snowpark.functions import col

# NOTE: these time window calculations are approximates and are not handling time gaps

def pre_aggregate_fn(df, ts_col, group_by_cols):
    df = df.with_column("WINDOW_END", F.call_udf("vec_window_end", F.col(ts_col), "15m"))
    df = df.group_by(group_by_cols + ["WINDOW_END"]).agg(
            F.sum("FARE_AMOUNT").alias("FARE_SUM_1_hr"),
            F.count("*").alias("TRIP_COUNT_1_hr")
         )
    return df

def pickup_features_fn(df):
    df = pre_aggregate_fn(df, "PICKUP_TS", ["PULOCATIONID"])
    
    window1 = Window.partition_by("PULOCATIONID").order_by(col("WINDOW_END").desc()).rows_between(Window.CURRENT_ROW, 7)
    window2 = Window.partition_by("PULOCATIONID").order_by(col("WINDOW_END").desc()).rows_between(Window.CURRENT_ROW, 19)

    df = df.with_columns(
        [
            "SUM_FARE_2_hr",
            "COUNT_TRIP_2hr",
            "SUM_FARE_5_hr",
            "COUNT_TRIP_5hr",
        ],
        [
            F.sum("FARE_SUM_1_hr").over(window1),
            F.sum("TRIP_COUNT_1_hr").over(window1),
            F.sum("FARE_SUM_1_hr").over(window2),
            F.sum("TRIP_COUNT_1_hr").over(window2),
        ]
    ).select(
        [
            col("PULOCATIONID"),
            col("WINDOW_END").alias("TS"),
            (col("SUM_FARE_2_hr") / col("COUNT_TRIP_2hr")).alias("MEAN_FARE_2_hr"),
            (col("SUM_FARE_5_hr") / col("COUNT_TRIP_5hr")).alias("MEAN_FARE_5_hr"),
        ]
    )
    return df

def dropoff_features_fn(df):
    df = pre_aggregate_fn(df, "DROPOFF_TS", ["DOLOCATIONID"])
    window1 = Window.partition_by("DOLOCATIONID").order_by(col("WINDOW_END").desc()).rows_between(Window.CURRENT_ROW, 7)
    window2 = Window.partition_by("DOLOCATIONID").order_by(col("WINDOW_END").desc()).rows_between(Window.CURRENT_ROW, 19)

    df = df.select(
        [
            col("DOLOCATIONID"),
            col("WINDOW_END").alias("TS"),
            F.sum("TRIP_COUNT_1_hr").over(window1).alias("COUNT_TRIP_2_hr"),
            F.sum("TRIP_COUNT_1_hr").over(window2).alias("COUNT_TRIP_5_hr"),
        ]
    )
    return df

pickup_df = pickup_features_fn(source_df)
pickup_df.show()

dropoff_df = dropoff_features_fn(source_df)
dropoff_df.show()

----------------------------------------------------------------------------------
|"PULOCATIONID"  |"TS"                 |"MEAN_FARE_2_HR"    |"MEAN_FARE_5_HR"    |
----------------------------------------------------------------------------------
|49              |2016-01-01 00:15:00  |7.0                 |7.0                 |
|49              |2016-01-01 00:30:00  |9.3                 |9.3                 |
|49              |2016-01-01 00:45:00  |9.409090909090908   |9.409090909090908   |
|49              |2016-01-01 01:00:00  |12.296296296296296  |12.296296296296296  |
|49              |2016-01-01 01:15:00  |13.540816326530612  |13.540816326530612  |
|49              |2016-01-01 01:30:00  |13.27027027027027   |13.27027027027027   |
|49              |2016-01-01 01:45:00  |13.145              |13.145              |
|49              |2016-01-01 02:00:00  |13.007936507936508  |13.007936507936508  |
|49              |2016-01-01 02:15:00  |13.00326797385621   |12.925806451612903  |
|49 

## Create FeatureViews and materialize

Once the FeatureView construction is done, we can materialize the FeatureView to the Snowflake backend and incremental maintenance will start.

In [122]:
pickup_fv = FeatureView(name="trip_pickup_time_series_features", entities=[trip_pickup], feature_df=pickup_df, timestamp_col="ts")
pickup_fv = fs.register_feature_view(feature_view=pickup_fv, version="v1", refresh_freq="1 minute", block=True)

In [123]:
dropoff_fv = FeatureView(name="trip_dropoff_time_series_features", entities=[trip_dropoff], feature_df=dropoff_df, timestamp_col="ts")
fs.register_feature_view(feature_view=dropoff_fv, version="v1", refresh_freq="1 minute", block=True)

FeatureView(_name=TRIP_DROPOFF_TIME_SERIES_FEATURES, _entities=[Entity(name=TRIP_DROPOFF, join_keys=['DOLOCATIONID'], desc=)], _feature_df=<snowflake.snowpark.dataframe.DataFrame object at 0x2ae0f3c40>, _timestamp_col=TS, _desc=, _query=SELECT "DOLOCATIONID", "WINDOW_END" AS "TS", sum("TRIP_COUNT_1_HR") OVER (PARTITION BY "DOLOCATIONID"  ORDER BY "WINDOW_END" DESC NULLS LAST  ROWS BETWEEN CURRENT ROW AND 7 FOLLOWING  ) AS "COUNT_TRIP_2_HR", sum("TRIP_COUNT_1_HR") OVER (PARTITION BY "DOLOCATIONID"  ORDER BY "WINDOW_END" DESC NULLS LAST  ROWS BETWEEN CURRENT ROW AND 19 FOLLOWING  ) AS "COUNT_TRIP_5_HR" FROM ( SELECT "DOLOCATIONID", "WINDOW_END", sum("FARE_AMOUNT") AS "FARE_SUM_1_HR", count(1) AS "TRIP_COUNT_1_HR" FROM ( SELECT "TRIP_DISTANCE", "FARE_AMOUNT", "PASSENGER_COUNT", "PULOCATIONID", "DOLOCATIONID", "PICKUP_TS", "DROPOFF_TS", vec_window_end("DROPOFF_TS", '15m') AS "WINDOW_END" FROM ( SELECT  *  FROM ( SELECT "TRIP_DISTANCE", "FARE_AMOUNT", "PASSENGER_COUNT", "PULOCATIONID", "DOL

## Explore FeatureViews
We can easily discover what are the materialized FeatureViews and the corresponding features with *__fs.list_feature_views()__*. 

We can also apply filters based on Entity name or FeatureView names.

In [124]:
fs.list_feature_views(entity_name="trip_pickup").select(["NAME", "VERSION", "ENTITIES", "FEATURE_DESC"]).show()

---------------------------------------------------------------------------------------------------------------------------------
|"NAME"                            |"VERSION"  |"ENTITIES"                                          |"FEATURE_DESC"             |
---------------------------------------------------------------------------------------------------------------------------------
|TRIP_PICKUP_TIME_SERIES_FEATURES  |V1         |[                                                   |{                          |
|                                  |           |  "{\"name\": \"TRIP_PICKUP\", \"join_keys\": [...  |  "MEAN_FARE_2_HR": null,  |
|                                  |           |]                                                   |  "MEAN_FARE_5_HR": null   |
|                                  |           |                                                    |}                          |
------------------------------------------------------------------------------------------

## Generate training data and train a model
The training data generation will lookup __point-in-time correct__ feature values and join with the spine dataframe.

In [125]:
spine_df = source_df.select(["PULOCATIONID", "DOLOCATIONID", "PICKUP_TS", "FARE_AMOUNT"])
training_data = fs.generate_dataset(
    spine_df=spine_df,
    features=[pickup_fv, dropoff_fv],
    materialized_table="yellow_tripdata_2016_01_training_data",
    spine_timestamp_col="PICKUP_TS",
    spine_label_cols = ["FARE_AMOUNT"]
)

training_data.df.show()
training_data.df.queries

-----------------------------------------------------------------------------------------------------------------------------------------------------------
|"PULOCATIONID"  |"DOLOCATIONID"  |"PICKUP_TS"          |"FARE_AMOUNT"  |"MEAN_FARE_2_HR"    |"MEAN_FARE_5_HR"    |"COUNT_TRIP_2_HR"  |"COUNT_TRIP_5_HR"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------
|132             |219             |2016-01-01 00:08:08  |20.0           |NULL                |NULL                |NULL               |NULL               |
|132             |219             |2016-01-01 00:28:28  |14.0           |37.342592592592595  |37.342592592592595  |NULL               |NULL               |
|164             |219             |2016-01-01 00:29:09  |55.0           |12.699115044247788  |12.699115044247788  |NULL               |NULL               |
|163             |219             |2016-01-01 00:31:08  |52.0   

{'queries': ['SELECT * FROM FS_TIME_SERIES_EXAMPLE.AWESOME_FS.yellow_tripdata_2016_01_training_data_2023_09_20_13_31_23'],
 'post_actions': []}

In [126]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

training_pd = training_data.df.to_pandas()
X = training_pd.drop(["FARE_AMOUNT", "PICKUP_TS"], axis=1)
y = training_pd["FARE_AMOUNT"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()

,PULOCATIONID,DOLOCATIONID,MEAN_FARE_2_HR,MEAN_FARE_5_HR,COUNT_TRIP_2_HR,COUNT_TRIP_5_HR
72133,211,68,9.953704,10.302292,564.0,1008.0
549762,79,225,10.022312,10.379619,13.0,38.0
129708,261,211,14.340517,14.800613,200.0,259.0
108349,100,161,14.896985,14.896985,305.0,305.0
474535,236,262,10.602582,10.602582,563.0,563.0


In [127]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
estimator = make_pipeline(imp, LinearRegression())

reg = estimator.fit(X, y)
r2_score = reg.score(X_test, y_test)
print(r2_score * 100,'%')

y_pred = reg.predict(X_test)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))

31.74351768372038 %
Mean squared error: 90.17


## Log model with Model Registry
We can log the model along with its training dataset metadata with model registry.

In [130]:
from snowflake.ml.registry import model_registry
import time

registry = model_registry.ModelRegistry(session=session, database_name="my_cool_registry", create_if_not_exists=True)

In [131]:
model_name = f"my_model_{time.time()}"

model_ref = registry.log_model(
    model_name=model_name,
    model_version="v1",
    model=estimator,
    dataset=training_data,
)

/opt/homebrew/anaconda3/envs/fs_demo/lib/python3.8/site-packages/snowflake/ml/model/model_signature.py:52: UserWarning: The sample input has 653271 rows, thus a truncation happened before inferring signature. This might cause inaccurate signature inference. If that happens, consider specifying signature manually.
  warnings.warn(


## Restore model and predict with latest features
We retrieve the training dataset from registry then construct dataframe of latest feature values. Then we restore the model from registry. At last, we can predict with latest feature values.

In [132]:
# Prepare some source prediction data
pred_df = training_pd.sample(3, random_state=996)[['PULOCATIONID', 'DOLOCATIONID', 'PICKUP_TS']]
pred_df = session.create_dataframe(pred_df)
pred_df = pred_df.select('PULOCATIONID', 'DOLOCATIONID', F.cast(pred_df.PICKUP_TS / 1000000, TimestampType()).alias('PICKUP_TS'))

/opt/homebrew/anaconda3/envs/fs_demo/lib/python3.8/site-packages/snowflake/snowpark/session.py:1833: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, ci_output = write_pandas(


In [133]:
# Enrich source prediction data with features
registered_training_data = registry.get_dataset(
    model_name="my_trained_model", 
    model_version="v1",
)

enriched_df = fs.retrieve_feature_values(
    spine_df=pred_df, 
    features=registered_training_data.load_features(), 
    spine_timestamp_col='PICKUP_TS'
).drop(['PICKUP_TS']).to_pandas()

In [134]:
model_ref = model_registry.ModelReference(
    registry=registry, 
    model_name=model_name, 
    model_version="v1"
).load_model()

pred = model_ref.predict(enriched_df)

print(pred)

[11.75684447  8.77725855 12.42049179]


## DO NOT READ
Below is a simple test for the window_end function

In [113]:
from snowflake.snowpark import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import functions as F, types as T
import datetime

session = Session.builder.configs(SnowflakeLoginOptions()).create()

udf_name = "window_end"
    
@F.pandas_udf(
    name=udf_name,
    replace=True,
    packages=["numpy", "pandas", "pytimeparse"],
    session=session,
)
def vec_window_end_compute(
    x: T.PandasSeries[datetime.datetime],
    interval: T.PandasSeries[str],
) -> T.PandasSeries[datetime.datetime]:
    import numpy as np
    import pandas as pd
    from pytimeparse.timeparse import timeparse

    time_slice = timeparse(interval[0])
    if time_slice is None:
        raise ValueError(f"Cannot parse interval {interval[0]}")
    time_slot = (x - np.datetime64('1970-01-01T00:00:00')) // np.timedelta64(1, 's') // time_slice * time_slice + time_slice
    return pd.to_datetime(time_slot, unit='s')

df = session.create_dataframe(
    [
        '2023-01-31 01:02:03.004',
        '2023-01-31 01:14:59.999',
        '2023-01-31 01:15:00.000',
        '2023-01-31 01:15:00.004',
        '2023-01-31 01:17:10.007',
    ], 
    schema=['a']
)
df = df.select([F.to_timestamp("a").alias("ts")])

df = df.select(["TS", F.call_udf(udf_name, F.col("TS"), "15m").alias("window_end")])
df.show()

ERROR:snowflake.snowpark._internal.server_connection:Failed to execute query [queryID: 01af19db-0406-b1b7-000c-a90273b47663] 
CREATE OR REPLACE 
TEMPORARY  FUNCTION  window_end(arg1 TIMESTAMP,arg2 STRING)
RETURNS TIMESTAMP
LANGUAGE PYTHON 
RUNTIME_VERSION=3.8

PACKAGES=('numpy','pandas','pytimeparse','cloudpickle==2.0.0')


HANDLER='compute'

AS $$
import pickle

func = pickle.loads(bytes.fromhex('8005955b030000000000008c17636c6f75647069636b6c652e636c6f75647069636b6c65948c0d5f6275696c74696e5f747970659493948c0a4c616d6264615479706594859452942868028c08436f6465547970659485945294284b024b004b004b074b054b434376640164006c007d02640164006c017d03640164026c026d037d0401007c047c016401190083017d057c0564006b087242740464037c01640119009b009d02830182017c007c02a0056404a10118007c02a00664056406a1021a007c051a007c0514007c0517007d067c036a077c06640664078d02530094284e4b008c0974696d6570617273659485948c1643616e6e6f7420706172736520696e74657276616c20948c13313937302d30312d30315430303a30303a3030944b018c0173948c04756e6

SnowparkSQLException: (1304): 01af19db-0406-b1b7-000c-a90273b47663: 002002 (42710): SQL compilation error:
Object 'WINDOW_END(ARG1 TIMESTAMP_NTZ, ARG2 VARCHAR):TIMESTAMP_NTZ(9)' already exists.

In [114]:
session.sql("select window_end(ts, '15m') from foobar").collect()

[Row(WINDOW_END(TS, '15M')=datetime.datetime(2023, 1, 31, 1, 15)),
 Row(WINDOW_END(TS, '15M')=datetime.datetime(2023, 1, 31, 1, 15)),
 Row(WINDOW_END(TS, '15M')=datetime.datetime(2023, 1, 31, 1, 30)),
 Row(WINDOW_END(TS, '15M')=datetime.datetime(2023, 1, 31, 1, 30)),
 Row(WINDOW_END(TS, '15M')=datetime.datetime(2023, 1, 31, 1, 30))]